In [ ]:
# define agent
def my_agent(obs, config):
    # Your code here: Amend the agent!

    ################################
    # Imports and helper functions #
    ################################
    import random
    import numpy as np 
    
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    
    import numpy as np
    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, piece, config):
        """
        col:=> col to which drop a ball 
        piece:=> which agent is dropping
        next_grid:=> grid after dropping the ball
        """
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = piece
        return next_grid

    # Returns True if dropping piece in column results in game win
    def check_winning_move(obs, config, col, piece):
        # Convert the board to a 2D grid
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)
        next_grid = drop_piece(grid, col, piece, config)
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[row,col:col+config.inarow])
                if window.count(piece) == config.inarow:
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(next_grid[row:row+config.inarow,col])
                if window.count(piece) == config.inarow:
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if window.count(piece) == config.inarow:
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if window.count(piece) == config.inarow:
                    return True
        return False

    #########################
    # Agent makes selection #
    #########################

    for col in valid_moves:
        if check_winning_move(obs, config, col, 1) or check_winning_move(obs, config, col, 2):
            # winning 
            return col
        
    return random.choice(valid_moves)

In [ ]:
# Test
from kaggle_environments import evaluate, make, utils
import numpy as np

def get_win_percentage(agent1, agent2, n_rounds= 100):
    # configuration of connect X 
    config = {"row": 6, "columns": 7, "inarow": 4}
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    outcomes += [[j,i] for (i,j) in  evaluate("connectx", [agent2, agent1], config, [], n_rounds//2)]
    print("Agent 1 win probability:", np.round(outcomes.count([1,-1])/len(outcomes),2))
    print("Agent 2 win probability:", np.round(outcomes.count([-1,1])/len(outcomes),2))
    
config = {"row": 6, "columns": 7, "inarow": 4}
get_win_percentage(my_agent,"negamax", 10)    
# env = make("connectx", debug=True)
# env.play([my_agent, None], width=500, height=450)

In [ ]:
# make submission
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(my_agent, "submission.py")